In [1]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from data.base_data_module import BaseDataset

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
# train_dataset = np.load("dataset/Train.npy", allow_pickle=True)
# test_dataset = np.load("dataset/Test.npy", allow_pickle=True)
# val_dataset = np.load("dataset/Val.npy", allow_pickle=True)

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)
test_keypoints = np.load("dataset/Test_keypoints.npy",  allow_pickle=True)
val_keypoints = np.load("dataset/Val_keypoints.npy",  allow_pickle=True)

train_mfccs = torch.load("dataset/Train_mfccs.pt" )
test_mfccs = torch.load("dataset/Test_mfccs.pt" )
val_mfccs = torch.load("dataset/Val_mfccs.pt" )

train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints)

print(train_mfccs_not_transp.shape)
train_set  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints)


train_loader = DataLoader(train_set)
# test_loader = DataLoader(test_dataset)
# val_loader = DataLoader(val_dataset)

torch.Size([80, 1600, 721])


In [3]:


train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)


train_mfccs = torch.load("dataset/Train_mfccs.pt" )


train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints)
train_set_nt  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints)

train_loader = DataLoader(train_set)
train_loader_nt = DataLoader(train_set_nt)


In [4]:
lstm_input_size = train_set[0][0].shape[0]

In [5]:
train_set.__len__()

80

In [41]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
# Here we define our model as a class
class LSTM(pl.LightningModule):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2, dnn_shape=136):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dnn_shape = dnn_shape

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(self.dnn_shape, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        #print(f"input.view(len(input) {input.view(len(input), self.batch_size, -1).shape}")
        # lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        lstm_out, self.hidden = self.lstm(input)
        #print(lstm_out.shape)
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        # y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        y_pred = self.linear(lstm_out)
        #print(f"y_pred.shape in net: {y_pred.shape}")
        return y_pred


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # scheduler = StepLR(optimizer, step_size=800, gamma=0.5)

        return optimizer

    def training_step(self, batch, batch_idx):
        X_batch, y_batch = batch
        #print(f" X_batch:{X_batch.shape}, y_batch: {y_batch.shape}")
        y_pred = self.forward(X_batch)
        #print(f"y_pred.shape: {y_pred.shape}, y_batch.shape(): {y_batch.shape}")
        loss_fn = torch.nn.MSELoss()
        loss = loss_fn(y_pred, y_batch.float())
        # criterion = nn.BCEWithLogitsLoss()
        # loss = criterion(y_pred, y_batch.unsqueeze(1))
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss}
        


    
    def validation_step(self, batch):
        X_batch, y_batch = batch
        y_pred = self.forward(X_batch)
        
        return {'loss': loss}

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        # scheduler = StepLR(optimizer, step_size=800, gamma=0.5)

        return optimizer

model = LSTM(1600, 136, batch_size=1, output_dim=136, num_layers=1)

In [40]:

if True:
    logger = pl.loggers.TensorBoardLogger("training/logs")


    trainer = pl.Trainer(logger=logger, weights_save_path="training/logs", gpus=1 )

    trainer.fit(model, train_loader)
    # trainer.test(model, datamodule=data)

t/s, loss=0.00411, v_num=60]y_pred.shape: torch.Size([1, 721, 136]), y_batch.shape(): torch.Size([1, 721, 136])
Epoch 1:  98%|█████████▊| 78/80 [00:03<00:00, 25.62it/s, loss=7.94e-05, v_num=60]
C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\pytorch_lightning\utilities\distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [13]:
#aaaaa

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)


train_mfccs = torch.load("dataset/Train_mfccs.pt" )


train_mfccs_not_transp = torch.load("dataset/Train_mfccs.pt" )
train_mfccs_not_transp = torch.transpose(train_mfccs_not_transp, 2, 1)

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints)
train_set_nt  = BaseDataset(data=train_mfccs_not_transp, targets=train_keypoints)

train_loader = DataLoader(train_set)
train_loader_nt = DataLoader(train_set_nt)


In [14]:
it = iter(train_loader)
first_mfcc, first_kp = next(it)
print( first_mfcc.shape, first_mfcc.size(-1))
print( first_kp.shape)

torch.Size([1, 721, 1600]) 1600
torch.Size([1, 721, 136])


In [15]:
first_kp[0][0]

tensor([0.2578, 0.5078, 0.2578, 0.5664, 0.2695, 0.6211, 0.2773, 0.6719, 0.2930,
        0.7148, 0.3320, 0.7617, 0.3594, 0.7812, 0.4141, 0.8047, 0.4805, 0.8281,
        0.5547, 0.8203, 0.5977, 0.7969, 0.6328, 0.7812, 0.6719, 0.7461, 0.6992,
        0.7031, 0.7148, 0.6484, 0.7383, 0.5977, 0.7539, 0.5391, 0.3164, 0.4805,
        0.3516, 0.4727, 0.3906, 0.4727, 0.4258, 0.4805, 0.4570, 0.4883, 0.5742,
        0.4922, 0.6055, 0.4883, 0.6406, 0.4883, 0.6797, 0.4883, 0.7070, 0.5000,
        0.5078, 0.5469, 0.5078, 0.5820, 0.5078, 0.6133, 0.5000, 0.6406, 0.4570,
        0.6484, 0.4805, 0.6562, 0.5000, 0.6641, 0.5234, 0.6641, 0.5391, 0.6562,
        0.3594, 0.5312, 0.3828, 0.5234, 0.4141, 0.5234, 0.4414, 0.5391, 0.4180,
        0.5469, 0.3828, 0.5469, 0.5742, 0.5469, 0.6055, 0.5391, 0.6328, 0.5391,
        0.6562, 0.5469, 0.6328, 0.5586, 0.6055, 0.5547, 0.4062, 0.7070, 0.4336,
        0.7031, 0.4805, 0.7031, 0.4922, 0.7031, 0.5156, 0.7031, 0.5547, 0.7070,
        0.5742, 0.7148, 0.5469, 0.7305, 

In [16]:
print(torch.reshape(first_kp[0,0], (1, 136)).squeeze()[0], torch.reshape(first_kp[0,0], (1, 136)).squeeze()[-1])
print(first_kp[0,0, 0],first_kp[0,0, -1] )

tensor(0.2578) tensor(0.7070)
tensor(0.2578) tensor(0.7070)


In [26]:
lstm = nn.LSTM(1600, 136, 1, batch_first=True)


In [27]:
print( first_mfcc.shape, first_mfcc.size(-1))


torch.Size([1, 721, 1600]) 1600


In [28]:
output, (hn, cn) = lstm(first_mfcc)

In [29]:
output.shape

torch.Size([1, 721, 136])

In [33]:
dl =  nn.Linear(136, 136)

y_pred = dl(output)

In [34]:
y_pred

tensor([[[-0.3615,  0.1972, -0.2867,  ...,  0.2887, -0.2519, -0.3320],
         [-0.4358,  0.2319, -0.4761,  ...,  0.4729,  0.1836, -0.2334],
         [-0.5979,  0.0870, -0.4274,  ...,  0.5176,  0.0870, -0.2092],
         ...,
         [ 0.0095, -0.0070, -0.0296,  ...,  0.0615, -0.0526,  0.0062],
         [ 0.0095, -0.0070, -0.0296,  ...,  0.0615, -0.0526,  0.0062],
         [ 0.0095, -0.0070, -0.0296,  ...,  0.0615, -0.0526,  0.0062]]],
       grad_fn=<AddBackward0>)